<a href="https://colab.research.google.com/github/sharadnatarajSJSU/Deep-Neural-Networks/blob/main/Part_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb --upgrade

In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2022-04-14 07:01:59--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2022-04-14 07:01:59--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [      <=>           ]  33.20M  27.8MB/s    in 1.2s    

2022-04-14 07:02:01 (27.8 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [3]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
sweep_config = {
    'method': 'random'
    }

In [5]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [6]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [9]:
parameters_dict.update({
    'epochs': {
        'value': 10}
    })

In [8]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [10]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 10},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [11]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: dahd3a4r
Sweep URL: https://wandb.ai/sharadnataraj/pytorch-sweeps-demo/sweeps/dahd3a4r


In [12]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [13]:
def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [15]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 9424kxez with config:
wandb: 	batch_size: 160
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.03735842865609246
wandb: 	optimizer: sgd


batch loss,█▄▄▃▃▃▂▂▃▃▂▂▂▁▂▂▂▂▂▂▂▁▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
batch loss,0.09057
epoch,9
loss,0.08705


wandb: Agent Starting Run: v9205joo with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.03370258358942583
wandb: 	optimizer: adam


batch loss,▅▂▅▁▂▅▂▃▄▆▃▂▂▂▅▂▃▄▄▃▄▂▃▄▄▄▄▇▂▃▃▃▅▄▅▄▃▇▃█
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▃▄▇▆▇▇▅
batch loss,1.05972
epoch,9
loss,1.39209


wandb: Agent Starting Run: j8e2pd1m with config:
wandb: 	batch_size: 48
wandb: 	dropout: 0.4
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.038181498942312646
wandb: 	optimizer: sgd


...

batch loss,█▆▄▅▄▃▂▃▂▁▂▄▁▂▂▃▄▃▃▆▃▁▂▁▃▂▃▄▃▃▂▄▂▄▃▁▂▂▄▂
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▂▂▂▂
batch loss,0.60303
epoch,9
loss,1.15548


wandb: Agent Starting Run: zq1cl321 with config:
wandb: 	batch_size: 48
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.04799848668287197
wandb: 	optimizer: adam


batch loss,▃▁▃▂▂▂▂▂▁▃▁▂▂▃▂▂▂▂▃▂▃▃▄▃▂▂▂▂█▂▂▃▂▃▂▃▃▃▂▂
epoch,▁▂▃▃▄▅▆▆▇█
loss,▅▁▄▆▆▆▆▆██
batch loss,3.48753
epoch,9
loss,2.01213


wandb: Agent Starting Run: 6dd6kmzu with config:
wandb: 	batch_size: 152
wandb: 	dropout: 0.4
wandb: 	epochs: 10
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.05737942715912813
wandb: 	optimizer: adam


batch loss,█▂▂▁▂▂▃▂▂▁▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▂▁▂▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
batch loss,1.42767
epoch,9
loss,1.41381
